### NERC dataset citations - DataCite
Code to collect NERC dataset DOIs and citation information from the DataCite API

In [1]:

import requests, time, json, re, datetime, os, sys
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import numpy as np
import pandas as pd
from math import ceil
from datetime import date
# from dataCite_fun import getDataCiteCitations_relationTypes, getPublicationInfo_timeCopy
# import exportCitationResultsToCsv
# from Results import convertCSVtoJSON


from citations_fun.getDataCiteCitations_relationTypes import getDataCiteCitations_relationTypes


## DataCite events API
Get citation event data from datacite

In [2]:
relation_type_id_list = ['is-cited-by', 'is-referenced-by', 'is-supplement-to', 'IsPartOf', 'IsContinuedBy', 'IsDescribedBy', 'IsDocumentedBy', 'IsDerivedFrom', 'IsRequiredBy']
dataCite_df_relationTypes = getDataCiteCitations_relationTypes(relation_type_id_list)

is-cited-by
Total records: 211
Total pages: 1
Status:  200
Page:  1
Final page
is-referenced-by
Total records: 2505
Total pages: 3
Status:  200
Page:  1
https://api.datacite.org/events?page%5Bcursor%5D=MTY3NTk0MDk2OTEyOCw4NTU1Nzg1NC00ZTBjLTRjZmMtODlkYS05ZDVmMTg0NjAzYzI&page%5Bsize%5D=1000&prefix=10.5285&relation-type-id=is-referenced-by
Status:  200
Page:  2
https://api.datacite.org/events?page%5Bcursor%5D=MTc0MTc5MzY1NzY3MSw2YTBmMWVhNi01NjFmLTQwN2YtYjllZC1iMWQxNzU0NmZlMTM&page%5Bsize%5D=1000&prefix=10.5285&relation-type-id=is-referenced-by
Status:  200
Page:  3
Final page
is-supplement-to
Total records: 15
Total pages: 1
Status:  200
Page:  1
Final page
IsPartOf
Total records: 0
Total pages: 0
IsContinuedBy
Total records: 0
Total pages: 0
IsDescribedBy
Total records: 0
Total pages: 0
IsDocumentedBy
Total records: 0
Total pages: 0
IsDerivedFrom
Total records: 0
Total pages: 0
IsRequiredBy
Total records: 0
Total pages: 0
Done!


In [ ]:
# join events data with dataset DOI metadata

### don't need to load file if done in same env as getNERCDataDOIs() ###
with open("results/nerc_datacite_dois.json") as f:
    nerc_datacite_dois = json.load(f)

nerc_datacite_dois_df = pd.DataFrame(nerc_datacite_dois)

datacite_doi_events_df = dataCite_df_relationTypes.merge(
    nerc_datacite_dois_df,
    left_on='data_doi',
    right_on='data_doi',
    how='left'  # left-join as dataCite_df_relationTypes is a subset of nerc_datacite_dois_df
)


datacite_doi_events_df_drop = datacite_doi_events_df.drop(['data_page_number', 'data_self_link'], axis = 1)

In [9]:
# output results file 
# today = date.today()
# dataCite_filename = "results/" + 'datacite_doi_events_df_retrieved_' + (today.strftime("%d%m%Y")) + '.csv'
datacite_doi_events_df_drop.to_csv("results/latest_results_dataCite.csv", index = False)

Don't need stuff below here:

In [ ]:
# don't need anymore as not using json?
key_to_column_mapping = {
    "publisher": "data_publisher",
    "data_doi": "data_doi",
    "title": "data_title",
    "authors": "data_authors",
    "publicationYear": "publicationYear",
    "dates": "dates",
    "source-id": "source-id",
    "relation-type-id": "relation-type-id",
    "pub_doi_url": "subj-id",
    "occurred-at": "occurred-at",
    "Page endpoint": "Page endpoint"
}

# need to delete datasetDOI_attribute at some point?
# renamed_json.pop('datasetDOI_attribute', None)


# # Rename the keys in merged_json according to the mapping
# renamed_json = {key_to_column_mapping.get(k, k): v for k, v in merged_json[0].items()}
# renamed_json

def rename_keys(dict, mapping):
    return {mapping.get(k, k): v for k, v in dict.items()}

renamed_json = [rename_keys(item, key_to_column_mapping) for item in merged_json]
datacite_json = renamed_json

    


In [ ]:
# # get dataset metadata - old - get rid?
# info_list = []
# headers = {'client-id': 'bl.nerc'}
# api_url = 'https://api.datacite.org/dois/' 
# # for doi in dataCite_df_relationTypes['data_doi']:
# for (source_id, relation_type_id, occurred_at, Page_endpoint, data_doi, pub_doi_url) in zip(dataCite_df_relationTypes['source-id'],dataCite_df_relationTypes['relation-type-id'],dataCite_df_relationTypes['occurred-at'],dataCite_df_relationTypes['Page endpoint'],dataCite_df_relationTypes['data_doi'], dataCite_df_relationTypes['pub_doi_url']):
#     r = requests.get((api_url + data_doi), headers)
#     print(r.status_code, data_doi)
    
#     try:
#         # process author info
#         author_list = []
#         for item in r.json()['data']['attributes']['creators']:
#             author_list.append(item['name'])

#         info_list.append([
#             r.json()['data']['attributes']['publisher'],
#             data_doi,
#             r.json()['data']['attributes']['titles'][0]['title'],
#             author_list,
#             r.json()['data']['attributes']['publicationYear'],
#             r.json()['data']['attributes']['dates'],
#             r.json()['data']['attributes']['registered'],
#             source_id, relation_type_id, pub_doi_url, occurred_at, Page_endpoint
#         ])
#     except Exception as e:
#         info_list.append(["error",data_doi,"error","error","error","error","error","error","error",pub_doi_url,"error","error"])
        
# columns = ['data_publisher', 'data_doi', 'data_title', 'data_authors', 'publicationYear', 'dates', 'registered', 
#            'source-id', 'relation-type-id', 'pub_doi_url', 'occurred-at', 'Page endpoint']
# dataCite_df = pd.DataFrame(info_list, columns = columns)    
# print("Done!")
    

In [ ]:
import importlib

importlib.reload(getPublicationInfo_timeCopy)